# Notebook to produce saved files to be used in the analysis


## Read files to produce list of galaxies

In [1]:
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import numpy as np
import pandas as pd
import random
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import scipy as sc
from  itertools import zip_longest
from astropy import constants

# read .fits file
image_file = get_pkg_data_filename('/home/ricardo/Documents/codes/Marvin_Manga/MaNGA_screening_test.fits')
fits.info(image_file)

galaxies = fits.getdata(image_file, ext=1)

#create list of plates and ifus
i = 0
plate = []
ifu = []

while i<len(galaxies):
    plate_list = galaxies[i][0]
    plate.append(plate_list)
    ifu_list = galaxies[i][1]
    ifu.append(ifu_list)
    i+=1

#save list of plates and ifus
data1 = np.column_stack([plate])  
datafile_path1 = f'plate.txt'
np.savetxt(datafile_path1 , data1, fmt= ['%d'])

data2 = np.column_stack([ifu])  
datafile_path2 = f'ifu.txt'
np.savetxt(datafile_path2 , data2, fmt= ['%s'])



Filename: /home/ricardo/Documents/codes/Marvin_Manga/MaNGA_screening_test.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1  Joined        1 BinTableHDU    413   2220R x 190C   [K, 32A, 32A, 32A, 32A, 32A, 32A, 32A, 32A, 32A, 32A, D, D, D, D, D, D, D, K, D, K, D, D, 53A, K, 32A, K, D, D, D, D, D, D, D, D, D, D, D, K, K, K, D, D, D, D, K, K, K, K, 32A, K, 19A, K, K, K, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, K, K, K, 6A, K, K, D, D, 7D, 7D, 7D, D, D, D, D, 7D, D, D, 7D, 7D, 7D, 7D, D, D, D, D, 7D, 7D, K, K, 12A, 10A, K, 4A, 30A, L, E, E, E, E, K, K, K, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 8A, 8A, 8A, 8A, 8A, K, K, 14A, 14A, 14A, 14A, 14A, 14A, 10A, E, 15A, 10A, K, E, 4E, 4E, E, E, 3E, 3E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 35E, 46E, 46E, 46E, 46E, 46E, E, E]   


## Create .txt with command to run in NIRVANA

In [2]:
def files_run(min_d, max_d, redux, analysis, daptype, rc, dc, snr=True):
    """
    Function that create list of commands to run on NIRVANA. 
    Creates 2 txt files: one to download the data and other to run.
    """
    plate_ifu_list_init = []

    data_sample_indx = range(min_d, max_d)

    axd = open('axisym_download.txt','w')
    f = open('axisym.txt','w')
    for i in data_sample_indx:
        plate_ifu_list_init.append(f'{plate[i]}-{ifu[i]}')
        print(f'nirvana_manga_download {plate[i]} {ifu[i]}    --redux {redux} --analysis {analysis} --dr DR17  --daptype {daptype}' ,\
          sep=' ', file=axd)
    axd.close()

    for i in data_sample_indx:
        if snr==False:
            print(f'nirvana_manga_axisym {plate[i]} {ifu[i]}  --redux {redux} --analysis {analysis} --dr DR17 \
          -t Stars --odir ./outputs/error/{plate[i]}-{ifu[i]} --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype}', sep=' ', file=f)
            print(f'nirvana_manga_axisym {plate[i]} {ifu[i]}  --redux {redux} --analysis {analysis} --dr DR17 \
         --odir ./outputs/error/{plate[i]}-{ifu[i]} --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype}' , sep=' ', file=f)
        elif snr==True:
            print(f'nirvana_manga_axisym {plate[i]} {ifu[i]}  --redux {redux} --analysis {analysis} --dr DR17 \
          -t Stars --odir ./outputs/error/snr15/{plate[i]}-{ifu[i]} --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype} --min_sig_snr=15 --min_vel_snr=15', sep=' ', file=f)
            print(f'nirvana_manga_axisym {plate[i]} {ifu[i]}  --redux {redux} --analysis {analysis} --dr DR17 \
         --odir ./outputs/error/snr15/{plate[i]}-{ifu[i]} --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype}  --min_sig_snr=15 --min_vel_snr=15' , sep=' ', file=f)

            
    f.close()
    return plate_ifu_list_init	 


In [3]:
red_sciama = '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/redux/v3_1_1/'
analysis_sciama =  '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/'
dap_sciama = 'HYB10-MILESHC-MASTARSSP'

red_lap = './redux/DR17/'
analysis_lap =  './analysis/DR17/'
dap_lap = 'HYB10-MILESHC-MASTARHC2'

# different velocity curves used
rc1 = 'HyperbolicTangent'
dc1 = 'Exponential'
rc2 = 'PolyEx'
dc2 = 'ExpBase'

# set outuput directories
oroot = '/home/ricardo/Documents/codes/NIRVANA/sciama/outputs/error'
outdir = '/home/ricardo/Documents/codes/NIRVANA/sciama'

plate_ifu_list_init = files_run(0,len(ifu),red_sciama,analysis_sciama, dap_sciama, rc1, dc1)

def read_files_pd(file_path):
        """
        function to read files
        """
        if Path(file_path).exists():
            data = pd.read_csv(file_path, sep='\s+',header=None)
            return pd.DataFrame(data)

def plate_ifu_list_final(tracer, function_rc, function_dc, vel_type, dap_type = dap_sciama):
    """
    Read the output files from Nirvana to see if the file already exists. 
    Then a final list of ID is produced
    """
    plate_ifu_list_final = []
    for plate_ifu in plate_ifu_list_init:
        if read_files_pd(f'{oroot}/{plate_ifu}/nirvana-manga-axisym-{plate_ifu}-{tracer}-{function_rc}-{function_dc}-{dap_type}-fit_err-{vel_type}.txt')  is not None: 
            plate_ifu_list_final.append(f'{plate_ifu}')
    return plate_ifu_list_final

plate_ifu_list = plate_ifu_list_final('Gas', 'HyperbolicTangent','Exponential', 'vel1vel2')


## Class that creates results from Nirvana

Produces circular velocity for stars and gas and the difference between them

In [4]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline  
import pandas as pd
import nirvana
import os
import glob
import re
from scipy import interpolate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from  itertools import zip_longest

red_sciama = '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/redux/v3_1_1/'
analysis_sciama =  '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/'
dap_sciama = 'HYB10-MILESHC-MASTARSSP'
dap_spx = 'SPX-MILESHC-MASTARSSP'

red_lap = './redux/DR17/'
analysis_lap =  './analysis/DR17/'
dap_lap = 'HYB10-MILESHC-MASTARHC2'
rc1 = 'HyperbolicTangent'
dc1 = 'Exponential'
rc2 = 'PolyEx'
dc2 = 'ExpBase'

oroot = '/home/ricardo/Documents/codes/NIRVANA/sciama/outputs/error'
outdir = '/home/ricardo/Documents/codes/NIRVANA/sciama'

class Vdiff2:
    def __init__(self, plate_ifu, step=0.1, func_rc='HyperbolicTangent', \
                 func_dc='Exponential', oroot=oroot, outdir=outdir, update=True):
        self.plate_ifu = plate_ifu
        self.step = step
        self.update = update
        self.func_rc = func_rc
        self.func_dc = func_dc
        self.oroot = oroot
        self.outdir = outdir

    def read_files_pd(self,file_path):
        """
        function to read files
        """
        if Path(file_path).exists():
            data = pd.read_csv(file_path, sep='\s+',header=None)
            return pd.DataFrame(data)

    
    def v_mean(self,xbf, ybf, ybf_err):
        """
        Function to bin the data, given a step, and the list of r and v
        """
        c = list(np.arange(0,max(xbf),self.step))
        r_list = []
        r_mean_list = []
        v_list_ri = []
        v_mean_list = []
        r_std_list = []
        v_std_list = []
        v_err_list = []
        v_err_mean_list = []

        # select a radius from bin list
        for ci in c:      
        # select a radius from the list of data
            for xbfi in xbf:
        # compare the radius from the data with a radius from the bin list
        # if between 0 and 1, e.g., store velocity and radius
                if xbfi < ci + self.step and xbfi >= ci:
                    r_index = list(xbf).index(xbfi)
                    v_list_ri.append(ybf[r_index])
                    r_list.append(xbfi)
                    v_err_list.append(ybf_err[r_index])

        # calculate mean of r and v
            r_mean_list.append(np.mean(r_list))
            v_mean_list.append(np.mean(v_list_ri))
            v_err_mean_list.append(np.sum(np.square(v_err_list))/(len(v_err_list)*len(v_err_list)) )

        # calculate standard deviation of r and v
            r_std_list.append(np.std(r_list))
            v_std_list.append(np.std(v_list_ri))
        # clear list because it will start new radius from list
            v_list_ri.clear()
            r_list.clear()
        # return the binned radius, velocity, velocity squared, std for radius,  std for velocity, 
        # std for measured velocity
        return r_mean_list, v_mean_list, np.square(v_mean_list), r_std_list, v_std_list, np.sqrt(v_err_mean_list)
        # A 'nan' will appear in the output if there is not data inside the bin, e.g. there is no 0.4 in radius, thus 
        # the first averaged data points from r=0 to r=0.5 is non-existent


    def v_diff2(self,vtot_gas, vtot_stars):
        """
        Function to calculate difference in quadrature of v_tot gas^2 - v_tot stars^2
        """
        return list([x-y for x,y in zip(vtot_gas, vtot_stars)])


    def v_sig_tot2(self,vgas, vstar, vdispgas, vdispstar):
        """
        Function to calculate error^2 
        """
        return list([np.square(x)+np.square(y)+np.square(w)+np.square(z) for x,y,w, z in zip(vgas, vstar, vdispgas,\
                                                 vdispstar)])

    def store_files(self,  tracer,   vel_type,   dap_type = dap_sciama):
        """
        Function to store variables
        """
        r_list = []
        r_list_std = []
        v_list = []
        v_list_std = []
        v_list_std_par = []
        v_list_std_exp = []
        r_mean = []
        v_mean_list = []
        v_mean_sq = []
        v_mean_std = []
        r_std = []
        v_std = []


        # store each data in one list called data_t
        globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'] = self.read_files_pd(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-{tracer}-{self.func_rc}-{self.func_dc}-{dap_type}-fit_err-{vel_type}.txt')   
         # read columns with radius and velocities
        r_list = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][0]
        v_list = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][1]
        if vel_type == 'disp_vel':
            v_list_std = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][2]

        else:
            # propagated uncertainty + err experiment rotation velocity 
            v_list_std = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][5]
            # propagated uncertainty radius
            r_list_std = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][2]
            # propagated uncertainty  rotation velocity
            v_list_std_par = globals()[f'data_{tracer}_{self.func_rc}-{self.func_dc}_{self.plate_ifu}'][3]

        image_file_g = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Gas-HyperbolicTangent-Exponential-{dap_sciama}.fits.gz')
        reff = fits.getdata(image_file_g, ext=23)['REFF'][0]

        r_mean, v_mean_list,v_mean_sq,r_std, v_std, v_mean_err = \
                    self.v_mean(r_list/reff, v_list, v_list_std  )         


        # second index is the data set. When using different lists r_mean, etc, use the same dataset (second index)       
        if vel_type == 'disp_vel':
            return r_mean, v_mean_list, v_mean_sq, r_std, v_std, r_list, v_list, v_list_std, v_mean_err
        else:
         # v_mean_err is the mean of the measure/determined uncertainty (both from experiment and propagated params)   
            return r_mean, v_mean_list, v_mean_sq, r_std, v_std, r_list, v_list, v_list_std, v_mean_err, r_list_std,\
                   v_list_std_par


    def v_diff2_models(self,z_mod1, z_mod2):
        """
        Calculate difference of velocity between different models.
        """
        return [np.absolute(x-y) for x,y in zip(z_mod1, z_mod2)]
        

    def nan_test(self, tracer,  vel_type):
        """
        Check if there are negative vrot_err^2 -> gives a 'nan'.  CORRECT
        """

        if len(np.where (np.isnan(store_files( tracer, vel_type )[9])))!=0 or \
           len(np.where (np.isnan(store_files(plate_ifu,tracer, func_rc, fucntion_dc, vel_type , 0.5)[10])))!=0:
            print(f'{plate_ifu}, {tracer}, {func_rc}, {function_dc}, {vel_type}')
            print(np.where (np.isnan(store_files(plate_ifu,tracer, function_rc,function_dc, vel_type , 0.5)[9])))
            print(np.where (np.isnan(store_files(plate_ifu,tracer, function_rc,function_dc, vel_type , 0.5)[10])))

    def sersicn(self):
        image_file_g =get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Gas-HyperbolicTangent-Exponential-{dap_sciama}.fits.gz')
        return fits.getdata(image_file_g, ext=23)['SERSICN'][0]

    def v2_tracer_final(self):
        """
        Calculate the sum of the dispersion velocity and rotation velocity for stars and gas

        """
        vrot2_g = self.store_files('Gas','vel1vel2')[2]
        vdisp2_g =  self.store_files('Gas', 'disp_vel')[2]
        vrot2_s = self.store_files( 'Stars','vel1vel2')[2]
        vdisp2_s =  self.store_files( 'Stars',  'disp_vel' )[2]

        vgas_std_err = self.store_files('Gas' ,'vel1vel2'  )[8]
        vstar_std_err = self.store_files( 'Stars' , 'vel1vel2')[8]
        disp_vgas_err = self.store_files( 'Gas',   'disp_vel' )[8]
        disp_vstar_err = self.store_files( 'Stars' , 'disp_vel' )[8]

        rgv = self.store_files( 'Gas',  'vel1vel2' )[0]
        rv = self.store_files( 'Stars', 'vel1vel2' )[0]
        rg = self.store_files( 'Gas',  'disp_vel' )[0]
        r = self.store_files( 'Stars',  'disp_vel' )[0]
        r_gf = [(x+y)/2 for x,y in zip(rgv, rg)]
        r_sf = [(x+y)/2 for x,y in zip(rv, r)]

        if  self.update == False:
            vgas2 = list([x+y for x,y in zip(vrot2_g,vdisp2_g)])
            vstar2 = list([x+y for x,y in zip(vrot2_s,vdisp2_s)])
            vg2_err = list([np.sqrt(4*x*y*y + 4*w*z*z) for  x,y, w,z in zip(vrot2_g, vgas_std_err, vdisp2_g, disp_vgas_err)])
            vs2_err = list([np.sqrt(4*x*y*y + 4*w*z*z) for  x,y, w,z in zip(vrot2_s, vstar_std_err, vdisp2_s, disp_vstar_err)])

            return vgas2, vstar2, vg2_err, vs2_err, r_gf, r_sf

        else:
            # for gas
            image_file_g = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Gas-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
            reff = fits.getdata(image_file_g, ext=23)['REFF'][0]
            hsigmagas = fits.getdata(image_file_g, ext=23)['F_S_H'][0]/reff
            vhsigmagas = fits.getdata(image_file_g, ext=23)['F_S_CEN'][0]
            err_hsigma_gas = fits.getdata(image_file_g, ext=23)['E_S_H'][0]/reff

            sigma2gas_1 = [4*(rg[i]/hsigmagas)* vdisp2_g[i] for i in range(len(rg))]
            sigma2gas_2 = [8*(rg[i]/hsigmagas)* vdisp2_g[i] for i in range(len(rg))]
            vgas2_1 = list([x+y for x,y in zip(vrot2_g,sigma2gas_1)])
            vgas2_2 = list([x+y for x,y in zip(vrot2_g,sigma2gas_2)])

            #for stars
            image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
            hsigma = fits.getdata(image_file_s, ext=23)['F_S_H'][0]/reff
            vhsigma = fits.getdata(image_file_s, ext=23)['F_S_CEN'][0]
            err_hsigma = fits.getdata(image_file_s, ext=23)['E_S_H'][0]/reff

            sigma2stars = [4*(r[i]/hsigma) * vdisp2_s[i] for i in range(len(r))]
            vstar2 = list([x+y for x,y in zip(vrot2_s,sigma2stars)])

            # errors star and gas there is actally no uncertainty on r for disp vel
            vs2_err = list([np.sqrt(4*x*y**2 + 4*a**2*z**2/w + a**2*err_hsigma**2/(hsigma**2)) for  x,y, w,z, a in \
                            zip(vrot2_s, vstar_std_err, vdisp2_s, disp_vstar_err, sigma2stars)])
            vg2_err_1 = list([np.sqrt(4*x*y**2 + 4*a**2*z**2/w +a**2*err_hsigma_gas**2/(hsigmagas**2)) for  x,y, w,z,a in \
                              zip(vrot2_g, vgas_std_err, vdisp2_g, disp_vgas_err, sigma2gas_1)])
            vg2_err_2 = list([np.sqrt(4*x*y**2 + 4*a**2*z**2/w+ a**2*err_hsigma_gas**2/(hsigmagas**2))  for  x,y, w,z,a in \
                              zip(vrot2_g, vgas_std_err, vdisp2_g, disp_vgas_err, sigma2gas_2)])
            return vgas2_1, vstar2, vgas2_2, vg2_err_1, vs2_err, vg2_err_2, r_gf, r_sf

    def v2_tracer_final_inter(self):
        """
        Calculate the sum of the dispersion velocity and rotation velocity for
        stars and gas, but using interpolation.
        """
        rgasrot_f = []
        vgasrot2_f = []
        rgasdisp_f = []
        vgasdisp2_f = []
        rgasrot = self.store_files( 'Gas','vel1vel2')[0]
        vgasrot2 = self.store_files( 'Gas','vel1vel2')[2]
        rgasdisp = self.store_files( 'Gas', 'disp_vel')[0]
        vgasdisp2  = self.store_files( 'Gas', 'disp_vel')[2]
        for a, b in zip(rgasrot,vgasrot2):
                if str(b) != 'nan':
                    rgasrot_f.append(a)
                    vgasrot2_f.append(b)
        for a, b in zip(rgasdisp,vgasdisp2):
                if str(b) != 'nan':
                    rgasdisp_f.append(a)
                    vgasdisp2_f.append(b)            
        vgasrot2_curve = sc.interpolate.interp1d(rgasrot_f,vgasrot2_f,fill_value="extrapolate")
        vgasdisp2_curve = sc.interpolate.interp1d(rgasdisp_f,vgasdisp2_f,fill_value="extrapolate")
        rtot_gas = sorted(set(rgasrot_f+ rgasdisp_f))
        if max(rgasrot_f)<max(rgasdisp):
            r_max_g = max(rgasrot_f) 
        else:
            r_max_g = max(rgasdisp_f)
        rtot_gas_f = [x for x in rtot_gas if x<= r_max_g]
        rstarsrot_f = []
        vstarsrot2_f = []
        rstarsdisp_f = []
        vstarsdisp2_f = []
        rstarsrot = self.store_files( 'Stars','vel1vel2')[0]
        vstarsrot2 = self.store_files( 'Stars','vel1vel2')[2]
        rstarsdisp = self.store_files( 'Stars', 'disp_vel')[0]
        vstarsdisp2  = self.store_files( 'Stars', 'disp_vel')[2]
        for a, b in zip(rstarsrot,vstarsrot2):
                if str(b) != 'nan':
                    rstarsrot_f.append(a)
                    vstarsrot2_f.append(b)
        for a, b in zip(rstarsdisp,vstarsdisp2):
                if str(b) != 'nan':
                    rstarsdisp_f.append(a)
                    vstarsdisp2_f.append(b)  
        vstarsrot2_curve = sc.interpolate.interp1d(rstarsrot_f,vstarsrot2_f,fill_value="extrapolate")
        vstarsdisp2_curve = sc.interpolate.interp1d(rstarsdisp_f,vstarsdisp2_f,fill_value="extrapolate")
        rtot_stars = sorted(set(rstarsrot_f+ rstarsdisp_f))
        if max(rstarsrot_f)<max(rstarsdisp_f):
            r_max_s = max(rstarsrot_f) 
        else:
            r_max_s = max(rstarsdisp_f)
        rtot_stars_f = [x for x in rtot_stars if x<= r_max_s]

        if max(rtot_gas_f)<max(rtot_stars_f):
            r_max = max(rtot_gas_f) 
        else:
            r_max = max(rtot_stars_f)
        rtot = sorted(set(rtot_gas_f+ rtot_stars_f))
        rtot_f = [x for x in rtot if x<= r_max]

        if  self.update == False:
            vgastot2 = [vgasrot2_curve(x) +vgasdisp2_curve(x) for x in rtot_gas_f] 
            vstarstot2 = [vstarsrot2_curve(x) +vstarsdisp2_curve(x) for x in rtot_stars_f] 
            vdiff2 = [vgasrot2_curve(x) +  vgasdisp2_curve(x) \
                  -vstarsrot2_curve(x) - vstarsdisp2_curve(x) for x in rtot_f]
            return rtot_gas_f, vgastot2, rtot_stars_f, vstarstot2, rtot_f, vdiff2 
        else:
            image_file_g = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Gas-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
            reff = fits.getdata(image_file_g, ext=23)['REFF'][0]
            hsigmagas = fits.getdata(image_file_g, ext=23)['F_S_H'][0]/reff
            vhsigmagas = fits.getdata(image_file_g, ext=23)['F_S_CEN'][0]


            vgastot2_1 = [vgasrot2_curve(x) +4*(x/hsigmagas)* vgasdisp2_curve(x) for x in rtot_gas_f]
            vgastot2_2 = [vgasrot2_curve(x) +8*(x/hsigmagas)* vgasdisp2_curve(x) for x in rtot_gas_f] 

            image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
            hsigma = fits.getdata(image_file_s, ext=23)['F_S_H'][0]/reff
            vhsigma = fits.getdata(image_file_s, ext=23)['F_S_CEN'][0]

            vstarstot2 = [vstarsrot2_curve(x) +4*(x/hsigma) * vstarsdisp2_curve(x) for x in rtot_stars_f]

            vdiff2_1 = [vgasrot2_curve(x) +4*(x/hsigmagas)* vgasdisp2_curve(x) \
                  -vstarsrot2_curve(x) -4*(x/hsigma) * vstarsdisp2_curve(x) for x in rtot_f]
            vdiff2_2 = [vgasrot2_curve(x) +8*(x/hsigmagas)* vgasdisp2_curve(x) \
                  -vstarsrot2_curve(x) -4*(x/hsigma) * vstarsdisp2_curve(x) for x in rtot_f]
            return rtot_gas_f, vgastot2_1, vgastot2_2, rtot_stars_f, vstarstot2, rtot_f, vdiff2_1, vdiff2_2


    def v_diff2_final(self): 
        """
        Calculate the difference squared of gas - stars.    
        """
        vdiff2_norm = []
        vdiff2_2_norm = []
        vdiff2_err_norm = []
        vdiff2_err2_norm = []
        if  self.update ==False:
            vgas2, vstar2, vgas2err, vstars2err, rg, rs = \
                    self.v2_tracer_final()
            vdiff2 = self.v_diff2(vgas2, vstar2)
            vdiff2err = list([np.sqrt(np.square(x)/np.square(z)+np.square(y)/np.square(w))*np.absolute(z)/np.absolute(w) for x,y,z,w in zip(vgas2err, vstars2err, vgas2, vstar2)])
            vdiff2err_f = [x for x in vdiff2err if str(x) != 'nan']
            # calculate normalized vdiff2
            for i in range(len(vdiff2)):
                vdiff2_norm.append(vdiff2[i]/vstar2[i])
            vdiff2_norm_f =  [x for x in vdiff2_norm if str(x) != 'nan']         
        else: 
            vgas2_1, vstar2,vgas2_2, vg2_err_1, vs2_err, vg2_err_2, rg, rs  = \
                        self.v2_tracer_final()

            vdiff2 = self.v_diff2(vgas2_1, vstar2)
            vdiff2_2 = self.v_diff2(vgas2_2, vstar2)
            vdiff2_err_1 =list([np.sqrt(np.square(x)/np.square(z)+np.square(y)/np.square(w))*np.absolute(z)/np.absolute(w) for x,y,z,w in zip(vg2_err_1, vs2_err,vgas2_1, vstar2)])
            vdiff2_err_2 =list([np.sqrt(np.square(x)/np.square(z)+np.square(y)/np.square(w))*np.absolute(z)/np.absolute(w) for x,y,z,w in zip(vg2_err_2, vs2_err,vgas2_2, vstar2)])
            vdiff2_err_1_f = [x for x in vdiff2_err_1 if str(x) != 'nan']
            vdiff2_err_2_f = [x for x in vdiff2_err_2 if str(x) != 'nan']

            for i in range(len(vdiff2)):
                vdiff2_norm.append(vdiff2[i]/vstar2[i]) 
            for i in range(len(vdiff2_2)):
                vdiff2_2_norm.append(vdiff2_2[i]/vstar2[i]) 
            vdiff2_norm_f =  [x for x in vdiff2_norm if str(x) != 'nan']  
            vdiff2_norm_f2 =  [x for x in vdiff2_2_norm if str(x) != 'nan']        
        rf = [(x+y)/2 for x,y in zip(rg, rs)]
        rff = [x for x in rf if str(x) != 'nan']
        if  self.update == False:
            return rff, vdiff2_norm_f, vdiff2err_f

        elif  self.update == True:
            return rff, vdiff2_norm_f, vdiff2_err_1_f,vdiff2_norm_f2, vdiff2_err_2_f


    def save_v2_tracer_tot(self,   dap_type = dap_sciama):
        """
        Save the results of the total circular velocity.
        """
        if  self.update == True:
            vgas2_1, vstar2, vgas2_2, vg2_err_1, vs2_err, vg2_err_2, r_gf, r_sf =\
                                self.v2_tracer_final()
            datag = np.column_stack([r_gf, vgas2_1, vg2_err_1, vgas2_2,vg2_err_1])
            datafile_pathg = f'{self.outdir}/nirvana-manga-vgas2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            datas = np.column_stack([r_sf, vstar2, vs2_err])
            datafile_paths = f'{self.outdir}/nirvana-manga-vstars2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            return np.savetxt(datafile_pathg , datag,  fmt = ['%10.4f', '%10.4f','%10.4f','%10.4f','%10.4f']),np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f','%10.4f'])
        else:
            vgas2_1, vstar2, vg2_err_1, vs2_err, r_gf, r_sf = self.v2_tracer_final()
            datag = np.column_stack([r_gf, vgas2_1, vg2_err_1])
            datafile_pathg = f'{self.outdir}/nirvana-manga-vgas2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            datas = np.column_stack([r_sf, vstar2, vs2_err])
            datafile_paths = f'{self.outdir}/nirvana-manga-vstars2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            return np.savetxt(datafile_pathg , datag,  fmt = ['%10.4f', '%10.4f','%10.4f']),np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f','%10.4f'])


    def save_diff2(self,  dap_type = dap_sciama):
        """
        Save the results of the difference of gas and stars.
        """
        if  self.update == True:
            _r, vdiff2, vsig2, vdiff2_2, vsig2_2 = self.v_diff2_final()
            data1 = np.column_stack([_r, vdiff2, vsig2,  vdiff2_2, vsig2_2])
            datafile_path1 = f'{self.outdir}/nirvana-manga-diff2-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            return np.savetxt(datafile_path1 , data1,  fmt = ['%10.4f', '%10.4f','%10.4f','%10.4f','%10.4f'])
        else:
            _r, vdiff2, vsig2 = self.v_diff2_final()
            data1 = np.column_stack([_r, vdiff2, vsig2])
            datafile_path1 = f'{self.outdir}/nirvana-manga-diff2-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            return np.savetxt(datafile_path1 , data1,  fmt = ['%10.4f', '%10.4f','%10.4f'])


    def save_diff2_inter(self, dap_type = dap_sciama):
        """
        Save the results of the difference of gas and stars, using interpolation.
        """
        if  self.update == True:
            rtot_gas_f, vgastot2_1, vgastot2_2, rtot_stars_f, vstarstot2, rtot_f, vdiff2_1, vdiff2_2\
                    = self.v2_tracer_final_inter() 
         
            data1 = np.column_stack([ rtot_f, vdiff2_1, vdiff2_2])
            datafile_path1 = f'{self.outdir}/nirvana-manga-diff2-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            datag = np.column_stack([rtot_gas_f, vgastot2_1, vgastot2_2])
            datafile_pathg = f'{self.outdir}/nirvana-manga-vgas2tot-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            datas = np.column_stack([rtot_stars_f, vstarstot2])
            datafile_paths = f'{self.outdir}/nirvana-manga-vstars2tot-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
            return np.savetxt(datafile_pathg , datag,  fmt = ['%10.4f', '%10.4f','%10.4f']),np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f']),np.savetxt(datafile_path1 , data1,  fmt = ['%10.4f', '%10.4f','%10.4f']) 

        else:
            rtot_gas_f, vgastot2, rtot_stars_f, vstarstot2, rtot_f, vdiff2 =  self.v2_tracer_final_inter() 
            data1 = np.column_stack([ rtot_f, vdiff2])
            datafile_path1 = f'{self.outdir}/nirvana-manga-diff2-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            datag = np.column_stack([rtot_gas_f, vgastot2])
            datafile_pathg = f'{self.outdir}/nirvana-manga-vgas2tot-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            datas = np.column_stack([rtot_stars_f, vstarstot2])
            datafile_paths = f'{self.outdir}/nirvana-manga-vstars2tot-inter-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt'
            return np.savetxt(datafile_pathg , datag,  fmt = ['%10.4f', '%10.4f']),np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f']),np.savetxt(datafile_path1 , data1,  fmt = ['%10.4f', '%10.4f']) 
    
           
    def data_quality_redchi2(self, min_chi2, max_chi2):
        image_file_g = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Gas-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
        rchi2g = fits.getdata(image_file_g, ext=23)['RCHI2'][0]
        
        image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
        rchi2s = fits.getdata(image_file_s, ext=23)['RCHI2'][0]
        
        if rchi2g<max_chi2 and rchi2g>min_chi2 and rchi2s<max_chi2 and rchi2s>min_chi2:
            keep_data_chi2.append(self.plate_ifu)
        else:
            pass
    
    def v2_rot_stars_asym(self, r, r_err, asymp=True):
        image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
        star_asymp = fits.getdata(image_file_s, ext=23)['F_V_ASYMP'][0]
        star_asymp_err = fits.getdata(image_file_s, ext=23)['E_V_ASYMP'][0]
        
        if asymp == False:
            reff = fits.getdata(image_file_s, ext=23)['REFF'][0]
            hsigma = fits.getdata(image_file_s, ext=23)['F_S_H'][0]/reff
            vhsigma = fits.getdata(image_file_s, ext=23)['F_S_CEN'][0]
            err_hsigma = fits.getdata(image_file_s, ext=23)['E_S_H'][0]/reff
            err_vhsigma = fits.getdata(image_file_s, ext=23)['E_S_CEN'][0]
            vdisp2_s = (vhsigma*np.exp(-r/hsigma))**2
            sigma2stars = 4*(r/hsigma) * vdisp2_s 
            vstar2 = star_asymp**2 + sigma2stars

            # errors star  ****** there is actually no uncertainty on r for disp vel
            term= vdisp2_s*(-4*r*(hsigma-r)/hsigma**3)**2
            vs2_err = np.sqrt((2*star_asymp*star_asymp_err)**2 +\
                             term*err_hsigma**2 + vdisp2_s*(4*r*err_vhsigma/(hsigma*vhsigma))**2)
        else:
            vstar2 = star_asymp**2
            vs2_err = (2*star_asymp*star_asymp_err)
        return vstar2, vs2_err
    
    def choose_ind(self):
        self.save_diff2() 
        self.save_v2_tracer_tot()
   

    def choose_ind_inter(self):
        self.save_diff2_inter() 



## Produce a list of the missing IDs,

which were not run yet, i.e. the outputs not produced yet.

In [5]:
"""
Produce a list of the missing IDs, which were not run yet, i.e. the outputs not produced yet.
"""
def vdiff2_list_final(func_rc, func_dc,dap_type):
        """
        Read the output files from this notebook to see if the file already exists. 
        Then a final list of ID is produced
        """
        vdiff2_list_missing = []
        for plate_ifu in plate_ifu_list:
            try:
                if Vdiff2(plate_ifu).read_files_pd(f'/home/ricardo/Documents/codes/NIRVANA/sciama/nirvana-manga-diff2-{plate_ifu}-{func_rc}-{func_dc}-{dap_type}-SumQuadUpdate.txt') is None: 
                    vdiff2_list_missing.append(f'{plate_ifu}')
            except:
                vdiff2_list_missing.append(f'{plate_ifu}')
        return vdiff2_list_missing

        
plate_ifu_missing = [x for x in plate_ifu_list_init if x not in plate_ifu_list]
print(f'Missing files from NIRVANA:{plate_ifu_missing}')
vdiff2_missing1 = vdiff2_list_final('HyperbolicTangent', 'Exponential', dap_sciama) 
print('RC HyperbolicTangent', vdiff2_missing1)
vdiff2_missing2 = vdiff2_list_final('PolyEx', 'ExpBase', dap_sciama)                                
print('RC PolyEx',vdiff2_missing2)



def files_run_missing(file, redux, analysis, daptype, rc, dc):
    """
    Function to produce the jobs missing.
    """
    b= []
    for ind in range(len(file)):
        a = file[ind]
        b.append(a.split("-"))

    axd = open('axisym_download_missing.txt','w')
    for i in range(len(b)):
          print(f'nirvana_manga_download {b[i][0]} {b[i][1]}    --redux {redux} --analysis {analysis} \
              --dr DR17  --daptype {daptype}' , sep=' ', file=axd)
    axd.close()    
    f = open('axisym_miss.txt','w')
    for i in range(len(b)):
        print(f'nirvana_manga_axisym {b[i][0]} {b[i][1]}  --redux {redux} --analysis {analysis} --dr DR17 \
          -t Stars --odir ./outputs/error/{b[i][0]}-{b[i][1]} --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype}', sep=' ', file=f)
        print(f'nirvana_manga_axisym {b[i][0]} {b[i][1]}  --redux {redux} --analysis {analysis} --dr DR17 \
          --odir ./outputs/error/{b[i][0]}-{b[i][1]}   --rc {rc} --dc {dc} --fisher --save_vrot --daptype {daptype}' , sep=' ', file=f)
                         
    f.close()
    

red_sciama = '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/redux/v3_1_1/'
analysis_sciama =  '/mnt/lustre/MaNGA/sas/dr17/manga/spectro/analysis/v3_1_1/3.1.0/'
dap_sciama = 'HYB10-MILESHC-MASTARSSP'
dap_spx = 'SPX-MILESHC-MASTARSSP'

red_lap = './redux/DR17/'
analysis_lap =  './analysis/DR17/'
dap_lap = 'HYB10-MILESHC-MASTARHC2'
rc1 = 'HyperbolicTangent'
dc1 = 'Exponential'
rc2 = 'PolyEx'
dc2 = 'ExpBase'


"""
Run to produce the list of jobs. Options are 'id_missing', 'vdiff_missing2' or 'vdiff_missing2'.
"""
plateifu_error = [x for x in plateifu_error]
plate_ifu_list_miss = files_run_missing(plateifu_error, red_sciama,analysis_sciama, dap_sciama,rc1,dc1)
#plate_ifu_list_miss = files_run_missing(vdiff2_missing2, red_sciama,analysis_sciama, dap_spx, rc2, dc2)


Missing files from NIRVANA:[]
RC HyperbolicTangent ['12495-12701', '12495-12702', '12495-12704', '12678-3701', '12678-6103', '12769-1902', '12769-3702', '8150-3703', '8155-12702', '8158-3702', '8562-12701', '8614-12705', '8936-1901', '8941-6101']
RC PolyEx ['10218-12702', '10226-9101', '10492-12704', '10496-12704', '10501-12702', '10501-12704', '10506-12702', '10507-12703', '10509-12704', '10511-12705', '10512-12701', '10512-9102', '10516-3701', '10842-3703', '10843-9102', '10845-12702', '11013-3701', '11018-3703', '11749-12701', '11749-12704', '11823-12702', '11865-12705', '11942-12704', '11952-1901', '11952-3702', '11956-12705', '11958-12703', '11962-6102', '11968-9102', '11974-12701', '11974-9102', '11982-12704', '11982-3703', '12066-12705', '12085-12704', '12088-6102', '12495-12701', '12512-12703', '12678-3701', '12678-6102', '12678-6103', '12769-1902', '12769-3702', '7981-12703', '8083-12702', '8133-12701', '8139-1901', '8148-1902', '8150-3703', '8154-6101', '8155-12702', '8158-37

NameError: name 'plateifu_error' is not defined

## Produce the difference of circular veloficities for all galaxies

and save files 

In [ ]:
plateifu_error = []
for ind in tqdm(plate_ifu_list):  
    try:
        #Vdiff2(ind, func_rc='PolyEx', func_dc='ExpBase').choose_ind()
        Vdiff2(ind).choose_ind()
    except:
        plateifu_error.append(ind)
        print(ind)
    

In [ ]:
len(plateifu_error)

In [ ]:
plateifu_error = [x for x in plateifu_error ]
plate_ifu_list_miss = files_run_missing(plateifu_error, red_sciama,analysis_sciama, dap_sciama,rc1,dc1)


## List of Sersic indices 

In [ ]:
# It depends only on the galaxy properties and not on the model for RC or DC, same result for differnt 
# func_rc and func_dc
small_sersicn = []
listerror = []
for i in tqdm(plate_ifu_list):
    try:
        if Vdiff2(i).sersicn()<6:
            small_sersicn.append(i)
    except:
        listerror.append(i)
    
print(len(small_sersicn))

## Class that reads and stores results produced by the previous class

In [5]:
class Vdiff2_store:
    def __init__(self, plate_ifu,  func_rc='HyperbolicTangent', \
                 func_dc='Exponential',  outdir=outdir, oroot=oroot, update=True):
        self.plate_ifu = plate_ifu
        self.update = update
        self.func_rc = func_rc
        self.func_dc = func_dc
        self.outdir = outdir
        self.oroot = oroot
        
    def store_results_v2_tracer_tot(self, dap_type = dap_sciama):
        """
        Read and store results produced in choose_ind()
        """
        
        if self.update == False:
            globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vgas2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt')  
            rg  = globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2g= globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_errg= globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt')  
            rs = globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2s=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_errs=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            return rg, v2g, v2_errg, rs, v2s, v2_errs
        else:
            globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vgas2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
            rg=globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2g=globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_errg=globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            v2_2g=globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][3]
            v2_err2g=globals()[f'datag_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][4]
            globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
            rs= globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2s=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_errs=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            return rg, v2g, v2_errg, v2_2g, v2_err2g, rs, v2s, v2_errs
        
        
    def v2tot_tracer_mean(self, min_r=0.5, max_r=3):
        """
        Mean over the radius for the difference squared of gas - stars.

        """
        
        if self.update == False:
            rg, v2g, v2_errg, rs, v2s, v2_errs = self.store_results_v2_tracer_tot()
            rg_f = [rg[i] for i in range(len(rg)) if rg[i]>=min_r and rg[i]<=max_r]
            rs_f = [rs[i] for i in range(len(rs)) if rs[i]>=min_r and rs[i]<=max_r]
            v2g_f = [v2g[i] for i in range(len(v2g)) if rg[i]>=min_r and rg[i]<=max_r]
            v2gerrf = [v2_errg[i] for i in range(len(v2_errg)) if rg[i]>=min_r and rg[i]<=max_r]
            v2gerrfsq =  [np.square(x) for x in zip(v2gerrf)]
            v2s_f = [v2s[i] for i in range(len(v2s)) if rs[i]>=min_r and rs[i]<=max_r]
            v2serrf = [v2_errs[i] for i in range(len(v2_errs)) if rs[i]>=min_r and rs[i]<=max_r]
            v2serrfsq =  [np.square(x) for x in zip(v2serrf)]
            
            # mean of uncertainties  is similar to standard deviation equation SD = sqrt (sum err^2/N)
            return np.nanmean(v2g_f), np.sqrt(np.nanmean(list(v2gerrfsq))), np.nanmean(v2s_f), np.sqrt(np.nanmean(list(v2serrfsq))),np.nanmean(rg_f),np.nanmean(rs_f)
        else:
            rg, v2g, v2_errg, v2_2g, v2_err2g, rs, v2s, v2_errs = self.store_results_v2_tracer_tot()
            rg_f = [rg[i] for i in range(len(rg)) if rg[i]>=min_r and rg[i]<=max_r]
            rs_f = [rs[i] for i in range(len(rs)) if rs[i]>=min_r and rs[i]<=max_r]
            v2g_f = [v2g[i] for i in range(len(v2g)) if rg[i]>=min_r and rg[i]<=max_r]
            v2gerrf = [v2_errg[i] for i in range(len(v2_errg)) if rg[i]>=min_r and rg[i]<=max_r]
            v2gerrfsq =  [np.square(x) for x in zip(v2gerrf)]
            v2g2_f = [v2_2g[i] for i in range(len(v2_2g)) if rg[i]>=min_r and rg[i]<=max_r]
            v2g2errf = [v2_err2g[i] for i in range(len(v2_err2g)) if rg[i]>=min_r and rg[i]<=max_r]
            v2g2errfsq =  [np.square(x) for x in zip(v2g2errf)]
            v2s_f = [v2s[i] for i in range(len(v2s)) if rs[i]>=min_r and rs[i]<=max_r]
            v2serrf = [v2_errs[i] for i in range(len(v2_errs)) if rs[i]>=min_r and rs[i]<=max_r]
            v2serrfsq =  [np.square(x) for x in zip(v2serrf)]

            return np.nanmean(v2g_f), np.sqrt(np.nanmean(list(v2gerrfsq))), np.nanmean(v2g2_f), np.sqrt(np.nanmean(list(v2g2errfsq))), np.nanmean(v2s_f), np.sqrt(np.nanmean(list(v2serrfsq))),np.nanmean(rg_f),np.nanmean(rs_f)
   

    def v2star_mean(self):
        """
        Mean over the radius for the Vcirc star, considering weighted mean.

        """
        rg, v2g, v2_errg, v2_2g, v2_err2g, rs, v2s, v2_errs = self.store_results_v2_tracer_tot()
        min_r = 0.5  
        rsff= []
        v2sff = []
        v2serrff= []

        for i in range(len(rs)):
            if str(rs[i]) == 'nan' or str(v2s[i]) == 'nan' or str(v2_errs[i]) == 'nan':
                pass
            else:
                rsff.append(rs[i])
                v2sff.append(v2s[i])
                v2serrff.append(v2_errs[i])
        max_r= max(rsff)        
        rs_f = [rsff[i] for i in range(len(rsff)) if rsff[i]>=min_r and rsff[i]<=max_r]


        v2s_f = [v2sff[i] for i in range(len(v2sff)) if rsff[i]>=min_r and rsff[i]<=max_r]
        v2serrf = [v2serrff[i] for i in range(len(v2serrff)) if rsff[i]>=min_r and rsff[i]<=max_r]
        v2serrfsq =  [np.square(x) for x in  v2serrf]


        ws  =  [1/x for x in v2serrfsq]
        return  np.average(v2s_f, weights=ws), 1/np.sqrt(np.average(ws))
        

    def store_results_vdiff2(self, dap_type = dap_sciama):
        """
        Read and store results produced in choose_ind()
        """
        
        if self.update == False:
            globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-diff2-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadrature.txt')  
            r  = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2 =globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_err = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            return r, v2, v2_err
        else:
            globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-diff2-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
            r  = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
            v2 = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
            v2_err = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
            v2_2 = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][3]
            v2_err2 = globals()[f'data_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][4]
            
            return r, v2, v2_err, v2_2, v2_err2

    def data_quality_accept(self, acceptance):
        if self.update==True:
            _rg, _v2g, _v2_errg, _v2_2g, _v2_err2g, _rs, _v2s, _v2_errs= \
                                self.store_results_v2_tracer_tot()

            v2s = [y for x,y in zip(_v2_errs,_v2s) if np.absolute(x/y) <acceptance]
            rs = [z for x,y,z in zip(_v2_errs,_v2s,_rs) if np.absolute(x/y) <acceptance]
            v2_errs =[x for x,y,z in zip(_v2_errs,_v2s,_rs) if np.absolute(x/y) <acceptance]
            
            _r_m, _v2, _v2err, _v22,_v22err = self.store_results_vdiff2()
            
            v2err = [_v2err[i] for i in range(len(_v2err)) if np.absolute(_v2err[i]/_v2[i]) <acceptance]
            v2 = [_v2[i] for i in range(len(_v2)) if np.absolute(_v2err[i]/_v2[i]) <acceptance]
            rm = [_r_m[i] for i in range(len(_r_m)) if np.absolute(_v2err[i]/_v2[i]) <acceptance]
            v22err = [_v22err[i] for i in range(len(_v22err)) if np.absolute(_v22err[i]/_v22[i]) <acceptance]
            v22 = [_v22[i] for i in range(len(_v22)) if np.absolute(_v22err[i]/_v22[i]) <acceptance]
            rm2 = [_r_m[i] for i in range(len(_r_m)) if np.absolute(_v22err[i]/_v22[i]) <acceptance]
            
            return rs,v2s,v2_errs,rm,v2,v2err,rm2,v22,v22err
        
 


    def data_quality_tracer(self, acceptance, thre):
        if self.update==True:
            _rg, _v2g, _v2_errg, _v2_2g, _v2_err2g, _rs, _v2s, _v2_errs= \
                                self.store_results_v2_tracer_tot()

            list_err_vstars = [x for x,y in zip(_v2_errs,_v2s) if np.absolute(x/y) <acceptance]
            threstar = thre*len(_v2s)
            if  len(list_err_vstars)>threstar:
                keep_data_list_star.append(self.plate_ifu)
            else:
                pass
        else:
            _rg, _v2g, _v2_errg, _rs, _v2s, _v2_errs= \
                                self.store_results_v2_tracer_tot()

            list_err_vstars = [x for x,y in zip(_v2_errs,_v2s) if np.absolute(x/y) <acceptance]
            threstar = thre*len(_v2s)
            if  len(list_err_vstars)>threstar:
                keep_data_list_star.append(self.plate_ifu)
            else:
                pass
        
    def data_quality(self, acceptance, thre):
        
        if self.update ==True:
            r_m, v2, v2err, v22,v22err = self.store_results_vdiff2()
        

            list_err_v1 = [np.absolute(v2err[i]/v2[i]) for i in range(len(v2err)) if np.absolute(v2err[i]/v2[i]) <acceptance]
            list_err_v2 = [np.absolute(v22err[i]/v22[i]) for i in range(len(v22err)) if np.absolute(v22err[i]/v22[i])<acceptance]
        
            thre1 = thre*len(v2)
            thre2 = thre*len(v22)

            if len(list_err_v1)>thre1:
                    keep_data_list.append(self.plate_ifu)
            else:
                 pass
            if len(list_err_v2)>thre2:
                    keep_data_list2.append(self.plate_ifu)
            else:
                 pass
        else:
            r_m, v2, v2err = self.store_results_vdiff2()
        

            list_err_v1 = [np.absolute(v2err[i]/v2[i]) for i in range(len(v2err)) if np.absolute(v2err[i]/v2[i]) <acceptance]
            
            thre1 = thre*len(v2)
            
            if len(list_err_v1)>thre1:
                    keep_data_list.append(self.plate_ifu)
            else:
                 pass
                   
            
    def vdiff2_mean(self, min_r=0.5, max_r=3):
        """
        Mean over the radius for the difference squared of gas - stars.

        """
        vdiff2_norm = []
        if self.update == False:

            r_m, v2, v2err = self.store_results_vdiff2()
            r_f = [r_m[i] for i in range(len(r_m)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2_f = [v2[i] for i in range(len(v2)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2errf = [v2err[i] for i in range(len(v2err)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2errfsq =  [np.square(x) for x in zip(v2errf)]

            return np.nanmean(v2), np.sqrt(np.nanmean(list(v2errfsq))*1/len(v2err)), np.nanmean(r_f)
        else:
            r_m, v2, v2err, v2_2, v2err2 = self.store_results_vdiff2()
            r_f = [r_m[i] for i in range(len(r_m)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2_f = [v2[i] for i in range(len(v2)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v22_f = [v2_2[i] for i in range(len(v2_2)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2errf = [v2err[i] for i in range(len(v2err)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2errfsq =  [np.square(x) for x in zip(v2errf)]
            v2err2f = [v2err2[i] for i in range(len(v2err)) if r_m[i]>=min_r and r_m[i]<=max_r]
            v2err2fsq =  [np.square(x) for x in zip(v2err2f)]

            return np.nanmean(v2_f), np.sqrt(np.nanmean(v2errfsq)),np.nanmean(v22_f),np.sqrt(np.nanmean(v2err2fsq)),np.nanmean(r_f)

    
    def v2_rot_stars_rs(self, r_err=0, dap_type=dap_sciama):
        image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
        reff = fits.getdata(image_file_s, ext=23)['REFF'][0]
        star_asymp = fits.getdata(image_file_s, ext=23)['F_V_ASYMP'][0]
        star_asymp_err = fits.getdata(image_file_s, ext=23)['E_V_ASYMP'][0]
        star_scale = fits.getdata(image_file_s, ext=23)['F_V_SCL'][0]/reff
        star_scale_err = fits.getdata(image_file_s, ext=23)['E_V_SCL'][0]/reff
        _,_,_,_,_, r,_,_=self.store_results_v2_tracer_tot()
        
        hsigma = fits.getdata(image_file_s, ext=23)['F_S_H'][0]/reff
        vhsigma = fits.getdata(image_file_s, ext=23)['F_S_CEN'][0]
        err_hsigma = fits.getdata(image_file_s, ext=23)['E_S_H'][0]/reff
        err_vhsigma = fits.getdata(image_file_s, ext=23)['E_S_CEN'][0]
        if self.func_rc =='HyperbolicTangent' and self.func_dc=='Exponential':
            vdisp2_s = (vhsigma*np.exp(-r/hsigma))**2
            sigma2stars = 4*(r/hsigma) * vdisp2_s
            vrot = star_asymp*np.tanh(r/star_scale)
            vstar2 = vrot**2 + sigma2stars

            # errors star  ****** there is actually no uncertainty on r for disp vel
            term=(- vdisp2_s*4*r*(hsigma-2*r)/hsigma**3)**2
            vs2_err = np.sqrt(4*vrot**2*star_asymp_err**2/star_asymp**2\
                              +4*vrot**2*star_asymp**2*r**2*star_scale_err**2/(star_scale**4*(np.cosh(r/star_scale))**4)\
                            +4*vrot**2*star_asymp**2*r_err**2/(star_scale**2*(np.cosh(r/star_scale))**4)\
                              +term*err_hsigma**2 + (8*vdisp2_s*r*err_vhsigma/(hsigma*vhsigma))**2)
        
        elif self.func_rc =='PolyEx' and self.func_dc=='ExpBase':
            alpha = fits.getdata(image_file_s, ext=23)['F_V_SLP'][0]
            alpha_err = fits.getdata(image_file_s, ext=23)['E_V_SLP'][0]
            base = fits.getdata(image_file_s, ext=23)['F_S_BASE'][0]
            base_err = fits.getdata(image_file_s, ext=23)['E_S_BASE'][0]
            
            vdisp2_s = (vhsigma*np.exp(-r/hsigma) + base)**2
            sigma2stars = 4*(r/hsigma) * vdisp2_s
            vrot = star_asymp*(1 - np.exp(-r/star_scale))* (1 + alpha*r/star_scale)
            vstar2 = vrot**2 + sigma2stars

            # errors star  ****** there is actually no uncertainty on r for disp vel
            term=(- 4*r*(hsigma*vdisp2_s-2*r*np.sqrt(vdisp2_s)*vhsigma*np.exp(-r/hsigma))/hsigma**3)**2
            vs2_err = np.sqrt(4*vrot**2*( (vrot*star_asymp_err/star_asymp)**2   \
                               + (star_asymp*(1 - np.exp(-r/star_scale))* (alpha*r/star_scale)*alpha_err)**2\
                               + ( star_asymp*(-r* np.exp(-r/star_scale)/star_scale**2)*(1 + alpha*r/star_scale)\
                                +star_asymp*(1 - np.exp(-r/star_scale))* (- alpha*r/star_scale**2) )**2*star_scale_err**2\
                                )\
                              +term*err_hsigma**2 + 8**2*vdisp2_s*r**2*err_vhsigma**2*np.exp(-2*r/hsigma)/hsigma**2\
                            + (8*r*base_err/hsigma)**2*vdisp2_s)
        
        datas = np.column_stack([r, vstar2, vs2_err])
        datafile_paths = f'{self.outdir}/nirvana-manga-vstars2tot_intrinsic-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt'
        return np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f','%10.4f']),np.savetxt(datafile_paths , datas,  fmt = ['%10.4f', '%10.4f','%10.4f'])
    
    
    def self_screen_par_rs(self,dap_type=dap_sciama):
        image_file_s = get_pkg_data_filename(f'{self.oroot}/{self.plate_ifu}/nirvana-manga-axisym-{self.plate_ifu}-Stars-{self.func_rc}-{self.func_dc}-{dap_sciama}.fits.gz')
        
        star_asymp = fits.getdata(image_file_s, ext=23)['F_V_ASYMP'][0]
        globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot_intrinsic-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
        rs = globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
        v2s=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
               
        return rs, [(star_asymp - x)/(constants.c.value/1000)**2 for x in v2s]
    
    def rnfw_fit(self, x, A):
        y = A/x**2
        return y    
    
    def rsRho_star_intrinsic(self, spheric,maxnfw_r, dap_type=dap_sciama,test=False, nfw=True,v2up=False):
        """
        Calculate the quantity r*rho, assuming cylindrical coordinates
        vc^2 = R d phi/d R
        Laplacian in cylindrical coordinates
        1/R d/dR(R *dphi/dR)/c^2 = -4piG/c^2 rho
        """
        globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot_intrinsic-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
        rsf = globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
        v2sf=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
        v2_errs=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
        rs = [x for x in rsf if str(x)!='nan']
        if v2up ==True:
            v2s = [x+y for x,y in zip(v2sf,v2_errs) if str(x)!='nan']
        else:
            v2s = [x for x in v2sf if str(x)!='nan']
        if len(rs)<2:
            return print(self.plate_ifu)
        else:
            pass
        
        if spheric ==False:
            rsV2s = [x/(constants.c.value/1000)**2 for x in v2s]
            deriv_rsv2s = np.absolute(np.diff(rsV2s)).tolist()
            
            if nfw == True:
                rs_nfwf = np.linspace(rs[-1], maxnfw_r, 100)
                rs_nfw = rs_nfwf[1:]               
                nfw = self.rnfw_fit(rs_nfw, deriv_rsv2s[-1]*rs[-1]).tolist()

                rs_tot = np.sort(list(set(rs[:-1] + rs_nfw.tolist())))
                rsrho_tot = deriv_rsv2s + nfw
                if test ==True:
                    return rs[:-1], rs_nfw,deriv_rsv2s, nfw
                else:
                    return rs_tot, rsrho_tot
            elif nfw==False:
                return rs, deriv_rsv2s
        else:
            rsV2s = [x*y/(constants.c.value/1000)**2 for x,y in zip(rs,v2s)]
            deriv_rsv2s = np.absolute(np.diff(rsV2s))
            rho = [x/(y) for x,y in zip(deriv_rsv2s,rs)]
            if nfw == True:
                rs_nfwf = np.linspace(rs[-1], maxnfw_r, 100)
                rs_nfw = rs_nfwf[1:]
                nfw = self.rnfw_fit(rs_nfw, rho[-1]*rs[-1]).tolist()

                rs_tot = np.sort(list(set(rs[:-1] + rs_nfw.tolist())))
                rsrho_tot = rho + nfw
                
                return rs_tot, rsrho_tot
            elif nfw==False:
                return rs, rho


    def Rho_star_intrinsic(self,spheric ,dap_type=dap_sciama):
        """
        Calculate the quantity rho, assuming cylindrical coordinates
        vc^2 = R d phi/d R
        Laplacian in cylindrical coordinates
        1/R d/dR(R *dphi/dR)/c^2 = -4piG/c^2 rho
        """
        globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot_intrinsic-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
        rs = globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
        v2s=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
        #v2_errs=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
        if spheric ==False:
            rsV2s = [x/(constants.c.value/1000)**2 for x in v2s]
            deriv_rsv2s = np.absolute(np.diff(rsV2s))
            rho = [x/y for x,y in zip(deriv_rsv2s, rs)]
            return rs, rho
        else:
            rsV2s = [x*y/(constants.c.value/1000)**2 for x,y in zip(rs,v2s)]
            deriv_rsv2s = np.absolute(np.diff(rsV2s))
            rho = [x/(y**2) for x,y in zip(deriv_rsv2s,rs)]
            return rs, rho
    
    
    
    def ratio_err_star_intrinsic(self,dap_type=dap_sciama):
        globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'] = Vdiff2.read_files_pd(self,file_path=f'{self.outdir}/nirvana-manga-vstars2tot_intrinsic-{self.plate_ifu}-{self.func_rc}-{self.func_dc}-{dap_type}-SumQuadUpdate.txt')  
        rs = globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][0]
        v2s=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][1]
        v2_errs=globals()[f'datas_{self.func_rc}_{self.func_dc}_{self.plate_ifu}'][2]
        ratio = [x/y for x, y in zip(v2_errs, v2s)]
        potential = [x/(constants.c.value/1000)**2 for x in v2s]
        return potential, ratio


 ## plot with error bars from experiment + parameter estimation

################################################################################################################
def plot_vel(plate_ifu, func_rc, func_dc, veltype, step, binned =False):
# plot of velocities - data points
    #if func == 'PolyEx' or func=='ExpBase':
    if veltype =='vel1vel2':
        c1 = 'b'
        c2 = 'green'
    else:
        c1 = 'r'
        c2 = 'orange'
    stars_d = plt.scatter(store_files(plate_ifu, 'Stars', func_rc,func_dc, veltype,  step)[5],\
                      store_files(plate_ifu,'Stars',func_rc,func_dc, veltype, step)[6], color=c1, marker='.', label='Stars data')
    gas_d = plt.scatter(store_files(plate_ifu,'Gas', func_rc,func_dc, veltype, step)[5], \
                    store_files(plate_ifu,'Gas',func_rc,func_dc, veltype, step)[6],color=c2,marker='.', label='Gas data')

    if binned == True:        
    # plot of velocities - binned
        stars_p= plt.scatter(store_files(plate_ifu,'Stars', func_rc,func_dc,veltype, step)[0], \
                     store_files(plate_ifu,'Stars', func_rc,func_dc, veltype, step)[1],color='indigo',marker='.', label='Stars binned')
        plt.errorbar(store_files(plate_ifu,'Stars', func_rc,func_dc, veltype,  step)[0], \
             store_files(plate_ifu,'Stars', func_rc,func_dc, veltype, step)[1], \
             yerr=  store_files(plate_ifu,'Stars', func_rc,func_dc,veltype, step)[8],  fmt='o')

        gas_p = plt.scatter(store_files(plate_ifu,'Gas', func_rc,func_dc, veltype, step)[0], \
                    store_files(plate_ifu,'Gas', func_rc,func_dc, veltype,  step)[1],color='orchid',marker='.', label='Gas binned')
        plt.errorbar(store_files(plate_ifu,'Gas',func_rc,func_dc, veltype,  step)[0], \
             store_files(plate_ifu,'Gas', func_rc,func_dc, veltype,  step)[1], \
             yerr=  store_files(plate_ifu,'Gas', func_rc,func_dc,veltype, step)[8], fmt='o')



    plt.yscale("log")
    plt.xlabel(r'$R$ (arcsec) ',fontsize=15)
    if veltype =='disp_vel':
        plt.ylabel(r'$\sigma V $ [km/s]',fontsize=15)
    else:
        plt.ylabel(r'$V $ [km/s]',fontsize=15)

    plt.legend()
    plt.title(plate_ifu)




def plot_v2diff(plate_ifu, func_rc, func_dc, update):
    c1 = 'darkslategrey'
    if update ==False:
        r_m, v2, v2err = store_results_vdiff2(plate_ifu, func_rc, func_dc, update)
        plt.scatter(r_m,v2,color=c1, marker='.', label = 'Vdiff')
        plt.errorbar(r_m, v2, yerr= v2err, fmt='o')
        rg, v2g, v2_errg,  rs, v2s, v2_errs=\
            store_results_v2_tracer_tot(plate_ifu, func_rc, func_dc,update, dap_type = dap_sciama)
        plt.scatter(rs,v2s,color=c1, marker='.', label = 'stars')
        plt.errorbar(rs,v2s, yerr= v2_errs, fmt='o')
    else:
        r_m, v2, v2err, v22, v2err2 = store_results_vdiff2(plate_ifu, func_rc, func_dc, update)
        rg, v2g, v2_errg, v2_2g, v2_err2g, rs, v2s, v2_errs = \
                store_results_v2_tracer_tot(plate_ifu, func_rc, func_dc,update=True, dap_type = dap_sciama)
#         plt.scatter(r_m,v2, marker='.', label = 'Vdiff2 4R/h',color='red')
#         plt.errorbar(r_m,v2, yerr= v2err, fmt='.',color='red')
#         plt.scatter(r_m,v22,  marker='.', label = 'Vdiff2 8R/h',color='green')
#         plt.errorbar(r_m,v22, yerr= v2err2, fmt='.',color='green')
        plt.scatter(rg,v2g,  marker='.', label = '4R/h',color='blue')
        plt.errorbar(rg,v2g, yerr= v2_errg, fmt='.',color='blue')
        plt.scatter(rs,v2s,  marker='.', label = 'stars',color='yellow')
        plt.errorbar(rs,v2s, yerr= v2_errs, fmt='.',color='yellow')
        plt.scatter(rg,v2_2g,  marker='.', label = '8R/h', color='cyan')
        plt.errorbar(rg,v2_2g, yerr= v2_err2g, fmt='.',color='cyan')



    plt.legend()
    plt.title(plate_ifu)
    plt.xlabel(r'$R$ ',fontsize=15)
    plt.ylabel(r'$  V^2$ ',fontsize=15)
    plt.title(plate_ifu)
    plt.yscale("symlog")


## Reduced chi2 Tanh & Exp

In [ ]:
keep_data_chi2 = []

for indx  in tqdm(small_sersicn):
    
    try:
      
            Vdiff2(indx).data_quality_redchi2(0.95, 1.05)
        
    except:
        pass
    
data1 = np.column_stack([keep_data_chi2])  
datafile_path1 = f'{outdir}/keep_data_chi2.txt'
np.savetxt(datafile_path1 , data1, fmt= ['%s'])


## Produce files with intrinsic stellar circular velocity

In [10]:
for ind in tqdm(plate_ifu_list):
    try:
        Vdiff2_store(ind).v2_rot_stars_rs()
    except:
        pass

100%|███████████████████████████████████████| 2220/2220 [23:40<00:00,  1.56it/s]


# Rotation curve - PolyEx // DC ExpBase

## Reduced chi2

In [ ]:
keep_data_chi2 = []

for indx  in tqdm(small_sersicn):
    
    try:
      
            Vdiff2(indx, func_rc='PolyEx', func_dc='ExpBase').data_quality_redchi2(0.95, 1.05)
        
    except:
        pass
    
data1 = np.column_stack([keep_data_chi2])  
datafile_path1 = f'keep_data_chi2_polyexp.txt'
np.savetxt(datafile_path1 , data1, fmt= ['%s'])

